### **1. ¿Qué es un cluster?**

Conjunto de máquinas (nodos) que trabajan juntas para procesar datos de manera distribuida.


### 2. Arquitectura de Spark en un Clúster
####  Componentes Clave en Spark


**a) Driver**: Programa principal que gestiona la ejecución de tareas.


**c) Nodo Maestro (Master Node)**: Es el nodo central del cluster. Solo existe en modo Standalone o Mesos. Si usas YARN o Kubernetes, este rol lo toma el gestor de recursos.

**d) Nodos Trabajadores (Worker Nodes)**: Ejecutan los cálculos.

**e) Ejecutores (Executors)**: Procesan los datos y ejecutan las tareas.

#### **a) Driver** (Dentro de Spark)

- Es el programa principal que ejecuta el código Spark.
- Contiene el SparkContext y envía tareas a los executors.
- Aquí actúa el Spark Scheduler, que planifica los Jobs y los divide en Stages y Tasks.

#### **b) Cluster Manager** (Fuera de Spark, administra los recursos)

- Es el gestor de recursos que asigna CPU, memoria y nodos.
- Puede ser YARN, Kubernetes, Mesos o el modo Standalone de Spark.
- El Spark Scheduler se comunica con el Cluster Manager para solicitar los recursos necesarios.

#### **c) Nodo Maestro** (Master Node)

- Es el nodo central del cluster.
- Solo existe en modo Standalone o Mesos.
- Si usas YARN o Kubernetes, este rol lo toma el gestor de recursos.

#### **d) Nodos Trabajadores** (Worker Nodes)

- Son las máquinas donde se ejecutan las tareas.
- Dentro de cada worker hay executors que procesan los datos.

#### **e) Executors** (Dentro de los Workers)

- Son los procesos que ejecutan las tareas.
- Cada executor tiene su propia memoria y CPU asignada.
- Devuelven los resultados al Driver cuando terminan.

#### Punto clave: 
En local (Docker o modo standalone) el driver y los executors están en la misma máquina, pero en un cluster real están distribuidos en varias máquinas. 

### 3. ¿Cuándo necesitamos un Clúster y qué nos aporta?
Un clúster es necesario cuando:

- Los datos son demasiado grandes para ser procesados en una sola máquina.

- Se requiere **escalabilidad** para manejar cargas de trabajo crecientes.

- Se necesita **paralelismo** para acelerar el procesamiento.

- Se desea**tolerancia a fallos** para garantizar la continuidad en caso de errores.

### 4. Modos de Ejecución de un Clúster
Spark soporta varios modos de ejecución en clúster:

- **Local**: Ejecución en una sola máquina, sin clúster.

- **Standalone**: Modo nativo de Spark, donde Spark gestiona sus propios recursos.

- **YARN**: Integración con Hadoop YARN para la gestión de recursos.

- **Mesos**: Uso de Apache Mesos como gestor de recursos.

- **Kubernetes**: Ejecución en contenedores gestionados por Kubernetes.

### 5. Configuraciones para Redes de Clústeres Escalables
Para garantizar la escalabilidad y eficiencia de un clúster, es importante considerar:

- **Particionamiento de datos**: Dividir los datos en particiones para distribuir la carga de trabajo.

- **Serialización**: Usar formatos eficientes como Kryo para reducir el tamaño de los datos transmitidos.

- **Uso eficiente de la memoria**: Ajustar la configuración de memoria para evitar desbordamientos y cuellos de botella.

### 6. Estrategias de Replicación
La replicación es clave para garantizar la tolerancia a fallos. Consiste en almacenar copias de los datos en múltiples nodos para evitar pérdidas en caso de fallos.

- **Importancia**: Asegura la disponibilidad y recuperación de datos en entornos distribuidos.


### 7. Monitorización de un Clúster
La monitorización es esencial para optimizar el rendimiento y detectar problemas:

- **UI de Spark**: Proporciona información detallada sobre el estado de las tareas, el uso de recursos y los tiempos de ejecución.

- **Logs**: Registros que ayudan a identificar errores y comportamientos inesperados.

### Ejercicio 1: Entender el Cluster y Verificar Nodos

**Objetivo**: Verificar cuántos nodos tiene el cluster y cuántos ejecutores están activos.


In [3]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("ClusterCheck").getOrCreate()

// Ver número de ejecutores activos
val executors = spark.sparkContext.getExecutorMemoryStatus.keys
println(s"Número de ejecutores disponibles: ${executors.size}")

// Ver detalles en localhost:4040
println("Consulta Spark UI en: http://localhost:4040")


Número de ejecutores disponibles: 1
Consulta Spark UI en: http://localhost:4040


spark = org.apache.spark.sql.SparkSession@78e35aa4
executors = Set(04842bb6bacb:37457)


Set(04842bb6bacb:37457)

#### Explicación

**getExecutorMemoryStatus.keys** obtiene la lista de ejecutores activos en el cluster.

**size** cuenta cuántos hay en total.

**Spark UI** permite ver el estado del cluster.

### Ejercicio 2: 

- Cargar y Procesar un Dataset en el Cluster
- **Objetivo**: Leer un archivo CSV y hacer una transformación en los datos.

Tenemos un archivo **ventas.csv** con el siguiente formato:

In [6]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("ProcesarVentas").getOrCreate()
val df = spark.read.option("header", "true").csv("ventas.csv")

// Calcular ventas totales por producto
val ventasTotales = df.withColumn("total", col("cantidad") * col("precio"))
ventasTotales.show()


+----------+----------+--------+------+------+
|     fecha|  producto|cantidad|precio| total|
+----------+----------+--------+------+------+
|2024-01-01|    Laptop|       2|  1200|2400.0|
|2024-01-02|Smartphone|       5|   800|4000.0|
|2024-01-03|    Tablet|       3|   600|1800.0|
+----------+----------+--------+------+------+



lastException = null
spark = org.apache.spark.sql.SparkSession@78e35aa4
df = [fecha: string, producto: string ... 2 more fields]
ventasTotales = [fecha: string, producto: string ... 3 more fields]


[fecha: string, producto: string ... 3 more fields]

#### Explicación
- read.csv() carga el archivo con encabezado.
- withColumn("total", col("cantidad") * col("precio")) crea una nueva columna con el total.
- .show() muestra los datos en consola.

### Ejercicio 3: Contar Palabras en un Cluster

**Objetivo**: Implementar el clásico Word Count usando RDDs en Spark.

Tenemos un **archivo txt** y ejecutams


In [8]:
val textFile = spark.sparkContext.textFile("texto.txt")

val wordCounts = textFile
  .flatMap(line => line.split(" "))
  .map(word => (word, 1))
  .reduceByKey(_ + _)

wordCounts.collect().foreach(println)

(Hola,2)
(Scala,1)
(Cluster,2)
(Spark,3)
(Datos,1)


textFile = texto.txt MapPartitionsRDD[44] at textFile at <console>:38
wordCounts = ShuffledRDD[47] at reduceByKey at <console>:43


ShuffledRDD[47] at reduceByKey at <console>:43

#### Explicación

**1.** textFile carga el archivo en un RDD.

**2.** flatMap divide el texto en palabras.

**3.** map convierte cada palabra en un par (palabra, 1).

**4.** reduceByKey(_ + _) suma los valores por cada palabra.

**5.** collect().foreach(println) imprime los resultados.

### Ejercicio 4: 
- Optimizar una Tarea en el Cluster
- **Objetivo**: Mostrar la diferencia entre lazy evaluation y persistencia.

#### Vamos a ejecutar el siguente código dos veces

In [12]:
val rdd = spark.sparkContext.parallelize(1 to 1000000)

// Sin persistencia
val result = rdd.map(x => x * 2).filter(_ % 3 == 0)
println(result.count())

333333


rdd = ParallelCollectionRDD[70] at parallelize at <console>:43
result = MapPartitionsRDD[72] at filter at <console>:46


MapPartitionsRDD[72] at filter at <console>:46

#### Luego, probar con persistencia:

In [14]:
import org.apache.spark.storage.StorageLevel

val rdd2 = spark.sparkContext.parallelize(1 to 1000000).persist(StorageLevel.MEMORY_ONLY)

val result2 = rdd2.map(x => x * 2).filter(_ % 3 == 0)
println(result2.count())


333333


rdd2 = ParallelCollectionRDD[73] at parallelize at <console>:45
result2 = MapPartitionsRDD[75] at filter at <console>:47


MapPartitionsRDD[75] at filter at <console>:47

#### Explicación

- En la primera versión, Spark recalcula todo cada vez.
- Con .persist(), los datos quedan en memoria y la segunda ejecución es más rápida.
- Se pueden ver las diferencias en localhost:4040.